In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome,
                           perform_experiments, ema_logging,Policy)

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

from problem_formulation import get_model_for_problem_formulation
from dike_model_function import DikeNetwork
from SALib.analyze import sobol

In [ ]:
dike_final_sobol = outcomes["Total Costs"]
dike_mean_sobol = np.mean(outcomes["Total Costs"])
dike_std_sobol = np.std(outcomes_sobol["Total Costs"])

In [ ]:
n_exp = 1000

results = perform_experiments(py_model, scenarios=n_exp, uncertainty_sampling=Samplers.LHS)

exp_lhs, out_lhs = results_lhs

In [ ]:
dike_final_sobol = outcomes_sobol["Total Costs"]
dike_mean_sobol = np.mean(outcomes_sobol["Total Costs"])
dike_std_sobol = np.std(outcomes_sobol["Total Costs"])

prey_final_sobol = outcomes_sobol['prey'][:,0,-1]
prey_mean_sobol = np.mean(outcomes_sobol['prey'][:,0,:],axis=1)
prey_std_sobol = np.std(outcomes_sobol['prey'][:,0,:],axis=1)

In [ ]:
scores = feature_scoring.get_ex_feature_scores(experiments_sobol, prey_final_sobol,
                                      max_features=0.6, mode=RuleInductionType.REGRESSION)[0]
scores

In [ ]:
combined_scores = []
for j in range(100, experiments_sobol.shape[0], 100):
    scores = feature_scoring.get_ex_feature_scores(experiments_sobol.iloc[0:j, :],
                                                   prey_mean_sobol[0:j],
                                                   max_features=0.6,
                                                   mode=RuleInductionType.REGRESSION)[0]
    scores.columns = [j]
    combined_scores.append(scores)

In [ ]:
combined_scores = pd.concat(combined_scores, axis=1, sort=True)

In [ ]:
fig, ax = plt.subplots(1)

combined_scores.T.plot(ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.set_xlabel('Samples')
ax.set_ylabel('feature scores')
plt.show()

In [ ]:
import seaborn as sns
cleaned_experiments = experiments_sobol.drop(columns=[l.name for l in dike_model.levers])

for policy in experiments_sobol.policy.unique():
    logical = experiments_sobol.policy == policy
    subset_results = {k:v[logical] for k,v in outcomes_sobol.items()}
    scores = feature_scoring.get_feature_scores_all(cleaned_experiments[logical],
                                                   subset_results)
    sns.heatmap(scores, annot=True, cmap='viridis')
    
    plt.title(f'Feature Scores for Policy: {policy}')

    plt.show()

In [ ]:
for policy in experiments_sobol.policy.unique():
    logical = experiments_sobol.policy == policy
    subset_results = {k: v[logical] for k, v in outcomes_sobol.items()}
    scores = feature_scoring.get_feature_scores_all(experiments_sobol[logical], subset_results)
    plt.figure(figsize=(14, 10))
    sns.heatmap(scores, annot=True, cmap='viridis')
    plt.title(f'Feature Scores for Policy: {policy}')
    plt.xlabel('Outcomes')
    plt.ylabel('Features')
    plt.show()

# Combined feature scoring for increasing sample sizes
combined_scores = []
dike_mean_sobol = outcomes_sobol['Total Costs'].mean(axis=1)

for j in range(100, experiments_sobol.shape[0], 100):
    scores = feature_scoring.get_ex_feature_scores(experiments_sobol.iloc[0:j, :],
                                                   dike_mean_sobol[0:j],
                                                   max_features=0.6,
                                                   mode=feature_scoring.RuleInductionType.REGRESSION)[0]
    scores.columns = [j]
    combined_scores.append(scores)

combined_scores = pd.concat(combined_scores, axis=1, sort=True)

fig, ax = plt.subplots(figsize=(14, 10))
combined_scores.T.plot(ax=ax)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xlabel('Samples')
ax.set_ylabel('Feature Scores')
plt.title('Combined Feature Scores Over Samples')
plt.show()